# Intervention Demo

<a target="_blank" href="https://colab.research.google.com/github/safety-research/circuit-tracer/blob/main/demos/intervention_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this demo, you'll learn how to perform interventions on models.

In [ ]:
#@title Colab Setup Environment

try:
    import google.colab
    !mkdir -p repository && cd repository && \
     git clone https://github.com/safety-research/circuit-tracer && \
     curl -LsSf https://astral.sh/uv/install.sh | sh && \
     uv pip install -e circuit-tracer/

    import sys
    from huggingface_hub import notebook_login
    sys.path.append('repository/circuit-tracer')
    sys.path.append('repository/circuit-tracer/demos')
    notebook_login(new_session=False)
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [1]:
from collections import namedtuple
from functools import partial

import torch 

from circuit_tracer import ReplacementModel

# display functions
from utils import display_topk_token_predictions, display_generations_comparison

First, we load our models (see `attribute_demo.ipynb` for more details).

In [2]:
model = ReplacementModel.from_pretrained("google/gemma-2-2b", "gemma", dtype=torch.bfloat16)

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b into HookedTransformer


We'll write some helper functions to print the top next tokens of our model, and a class to store features in.

In [3]:
Feature = namedtuple('Feature', ['layer', 'pos', 'feature_idx'])

# a display function that needs the model's tokenizer
display_topk_token_predictions = partial(display_topk_token_predictions, tokenizer=model.tokenizer)

## Example: Changing languages with zero ablations

Imagine that you have the following [annotated attribution graph](https://www.neuronpedia.org/gemma-2-2b/graph?slug=gemma-michael-jordan-es&clerps=%5B%5B%222308855%22%2C%22sports%22%5D%2C%5B%222502222%22%2C%22Spanish+articles%22%5D%2C%5B%222513416%22%2C%22Spanish%22%5D%2C%5B%222509334%22%2C%22Spanish%22%5D%2C%5B%222413490%22%2C%22Spanish%22%5D%2C%5B%222403018%22%2C%22Spanish%22%5D%2C%5B%222407980%22%2C%22Spanish+articles%22%5D%2C%5B%222511463%22%2C%22Spanish%22%5D%2C%5B%222104818%22%2C%22basketball%22%5D%2C%5B%222109324%22%2C%22sports%22%5D%2C%5B%222009090%22%2C%22basketball%22%5D%2C%5B%221712431%22%2C%22sports%22%5D%2C%5B%221515208%22%2C%22play%22%5D%2C%5B%22401305%22%2C%22game%22%5D%2C%5B%22109339%22%2C%22a%2Fal+in+Spanish%22%5D%2C%5B%2213978%22%2C%22romance+languages%22%5D%2C%5B%2215822%22%2C%22romance+languages%22%5D%2C%5B%221404939%22%2C%22play%22%5D%2C%5B%221915763%22%2C%22sports%22%5D%2C%5B%221812672%22%2C%22basketball%22%5D%2C%5B%221414510%22%2C%22sports%22%5D%2C%5B%22401742%22%2C%22basketball%22%5D%2C%5B%22101173%22%2C%22basketball%22%5D%2C%5B%22411%22%2C%22famous+people+%2F+named+entities%22%5D%2C%5B%222000341%22%2C%22Spanish%22%5D%2C%5B%222303604%22%2C%22sports+%2F+table+tennis+%2F+pool+%22%5D%2C%5B%222413277%22%2C%22%28incomprehensible%29%22%5D%5D&pinnedIds=27_143831_6%2C25_13416_6%2C24_3018_6%2C25_9334_6%2C24_13490_6%2C25_2222_6%2C24_7980_6%2C25_11463_6%2C21_9324_6%2C21_4818_6%2C23_8855_6%2C20_9090_6%2C17_12431_6%2C15_15208_6%2C14_4939_6%2C4_1305_6%2C1_9339_6%2CE_113501_5%2C0_13978_5%2C0_15822_5%2CE_717_6%2C19_15763_6%2C18_12672_6%2C4_1742_4%2C14_14510_4%2C1_1173_4%2CE_18853_4%2CE_7939_3%2C0_411_4%2C20_341_6&supernodes=%5B%5B%22basketball%22%2C%2220_9090_6%22%2C%2218_12672_6%22%2C%2221_4818_6%22%2C%2223_8855_6%22%5D%2C%5B%22sports%22%2C%2217_12431_6%22%2C%2219_15763_6%22%2C%2221_9324_6%22%5D%2C%5B%22play%22%2C%224_1305_6%22%2C%2214_4939_6%22%2C%2215_15208_6%22%5D%2C%5B%22basketball%22%2C%224_1742_4%22%2C%221_1173_4%22%5D%2C%5B%22romance+language%22%2C%221_9339_6%22%2C%220_15822_5%22%2C%220_13978_5%22%5D%2C%5B%22Spanish%22%2C%2225_9334_6%22%2C%2225_13416_6%22%2C%2224_13490_6%22%2C%2224_7980_6%22%2C%2224_3018_6%22%2C%2225_2222_6%22%2C%2225_11463_6%22%2C%2220_341_6%22%5D%5D&clickedId=20_341_6) showing the circuit for the Spanish sentence *Hecho: Michael Jordan juega al*, or in English, *Fact: Michael Jordan plays*. The correct answer, which the model correctly predicts, is *baloncesto*, or *basketball*. We observe a supernode of features that correspond to the Spanish language. Can we intervene on these features to change the model's output?

<img src="https://raw.githubusercontent.com/safety-research/circuit-tracer/main/demos/img/gemma/mj-basketball-es.png" width="400">

First, we can try to do this by identifying these supernode features, which we store below. For each, we store their layer, position (here, always -1, as all of these features are active at the final position), and feature ID. For the sake of convenience, we'll only add one supernode feature.

In [4]:
supernode_features = [
    Feature(layer=20,pos=-1,feature_idx=341),
]

Next, we need to turn our supernode features into a list of intervention tuples. These tuples are formatted as (layer, node, feature_idx, new_feature_value). For now, let's try just zeroing out these features at the last position.

In [5]:
intervention_tuples = [(*supernode_feature, 0.0) for supernode_feature in supernode_features]

Finally, we can run the intervention and view its effects on the model's output!

In [6]:
s = "Hecho: Michael Jordan juega al"

with torch.inference_mode():
    original_logits = model(s)
    new_logits, _ = model.feature_intervention(s, intervention_tuples)

display_topk_token_predictions(s, original_logits, new_logits)

Token,Probability,Distribution
baloncesto,0.617,61.7%
golf,0.051,5.1%
fútbol,0.047,4.7%
bás,0.044,4.4%
béisbol,0.034,3.4%
Token,Probability,Distribution
baloncesto,0.594,59.4%
golf,0.059,5.9%
bás,0.055,5.5%
fútbol,0.038,3.8%


That wasn't very effective! We do see that the probability of *basketball* has risen, bringing it into the top 5. But intervening on just one feature isn't enough to change the model's behavior dramatically; the rest of the distribution remains more or less the same. This is because many Spanish features contribute to our model's output language, while we have changed only one. Try changing more!

## Example: Swapping languages by turning features on

In the last example, we only turned Spanish features off, yielding text in English, which seems to be the model's default. But what if we wanted to swap to another language? 
Then we'd have to turn language features from that language on. Let's try this with another language, French. Here is the attribution graph for the analogous French sentence, [`Fait: Michael Jordan joue au` →`basket`](https://www.neuronpedia.org/gemma-2-2b/graph?slug=gemma-basket&clickedId=17_10566_2&clerps=%5B%5B%222308855%22%2C%22basketball%22%5D%2C%5B%222502222%22%2C%22Spanish+articles%22%5D%2C%5B%222513416%22%2C%22Spanish%22%5D%2C%5B%222104818%22%2C%22basketball%22%5D%2C%5B%222109324%22%2C%22sports%22%5D%2C%5B%222009090%22%2C%22basketball%22%5D%2C%5B%221712431%22%2C%22sports%22%5D%2C%5B%221515208%22%2C%22play%22%5D%2C%5B%22401305%22%2C%22game%22%5D%2C%5B%2213978%22%2C%22romance+languages%22%5D%2C%5B%2215822%22%2C%22romance+languages%22%5D%2C%5B%221404939%22%2C%22play%22%5D%2C%5B%221915763%22%2C%22sports%22%5D%2C%5B%221812672%22%2C%22basketball%22%5D%2C%5B%221414510%22%2C%22sports%22%5D%2C%5B%22401742%22%2C%22basketball%22%5D%2C%5B%22101173%22%2C%22basketball%22%5D%2C%5B%22411%22%2C%22famous+people+%2F+named+entities%22%5D%2C%5B%221710566%22%2C%22French%22%5D%5D&pinnedIds=27_12220_7%2CE_18853_5%2C21_4818_7%2C21_9324_7%2C23_3604_7%2C25_14882_7%2C24_15306_7%2C23_15317_7%2C20_9090_7%2C24_3329_7%2C19_15763_7%2C18_12672_7%2C17_12431_7%2C17_5253_7%2C15_15208_7%2C14_4939_7%2C6_7377_7%2CE_78224_6%2C4_1305_7%2C3_305_7%2C24_2086_7%2C24_3772_7%2C21_16354_7%2C20_1454_7%2C23_2592_7%2C22_10566_7%2C23_2554_7%2C17_10566_6%2C0_4076_6%2C14_14575_6%2C7_11689_6%2C4_1742_5%2C1_1173_5%2CE_7939_4&supernodes=%5B%5B%22game%2Fplay%22%2C%223_305_7%22%2C%224_1305_7%22%2C%226_7377_7%22%2C%2215_15208_7%22%2C%2214_4939_7%22%5D%2C%5B%22French%22%2C%220_4076_6%22%2C%227_11689_6%22%2C%2214_14575_6%22%2C%2217_10566_6%22%5D%2C%5B%22basketball%22%2C%2221_4818_7%22%2C%2218_12672_7%22%5D%2C%5B%22sports%22%2C%2217_12431_7%22%2C%2217_5253_7%22%2C%2221_9324_7%22%2C%2220_9090_7%22%2C%2219_15763_7%22%2C%2223_3604_7%22%2C%2223_15317_7%22%5D%2C%5B%22basketball%22%2C%224_1742_5%22%2C%221_1173_5%22%5D%2C%5B%22French%22%2C%2224_3329_7%22%2C%2221_16354_7%22%2C%2220_1454_7%22%2C%2223_2592_7%22%2C%2223_2554_7%22%2C%2224_2086_7%22%2C%2224_15306_7%22%2C%2225_14882_7%22%2C%2224_3772_7%22%2C%2222_10566_7%22%5D%5D). 

<img src="https://raw.githubusercontent.com/safety-research/circuit-tracer/main/demos/img/gemma/mj-basketball-fr.png" width="400">

The answer to the French query is "basket". Can we change that to Spanish? We start by taking one relatively low-level French feature, that feeds into all of the others.

In [7]:
french_supernode_features = [Feature(layer=20,pos=-1,feature_idx=1454)]

But what should we set the values of the French supernode features to be? Ideally, we set them to some in-distribution values. To do this, we can get the activations of these nodes on the French input sentence. We'll get these as a sparse tensor, to save on memory.

In [8]:
s_spanish = "Hecho: Michael Jordan juega al"
_, activations = model.get_activations(s_spanish, sparse=True)

Now, we construct and perform the intervention! Each supernode_feature contains precisely the information needed to index into `activations`.

In [9]:
spanish_supernode_features = supernode_features  # from before
fr_es_intervention_tuples = [(*supernode_feature, 0.0) for supernode_feature in french_supernode_features] 
fr_es_intervention_tuples+= [(*supernode_feature, 10*activations[supernode_feature]) for (supernode_feature) in spanish_supernode_features]

In [10]:
s_french = "Fait: Michael Jordan joue au"

with torch.inference_mode():
    original_logits = model(s_french)
    new_logits, _ = model.feature_intervention(s_french, fr_es_intervention_tuples)

display_topk_token_predictions(s_french, original_logits, new_logits)

Token,Probability,Distribution
basket,0.547,54.7%
basketball,0.117,11.7%
golf,0.065,6.5%
baseball,0.044,4.4%
football,0.033,3.3%
Token,Probability,Distribution
baloncesto,0.277,27.7%
golf,0.194,19.4%
fútbol,0.057,5.7%
basketball,0.047,4.7%


## Example: Interventions + Sampling
We've now intervened twice on the last token of the sentence; interventions on other positions work analogously. But what if we want to intervene in an open-ended fashion, allowing our model to generate tokens with that intervention still active? We can do this as follows, by setting the position of our intervention to an open-ended slice: `slice(pos, None, None)`. We set `pos` to be the last token of the original input, but you can also set it to an earlier position.

In [11]:
sequence_length = len(model.tokenizer(s_spanish).input_ids)
original_feature_pos = sequence_length - 1
open_ended_slice = slice(original_feature_pos, None, None)
open_ended_es_fr_intervention_tuples = [(layer, open_ended_slice, feature_idx, 0.0) for (layer, _, feature_idx) in spanish_supernode_features] 
open_ended_es_fr_intervention_tuples+= [(layer, open_ended_slice, feature_idx, 10*activations[layer, orig_pos, feature_idx]) for (layer, orig_pos, feature_idx) in spanish_supernode_features]

Now, we just get the necessary hooks by calling `model._get_feature_intervention_hooks`, and use those hooks to generate! Make sure to set `use_past_kv_cache` to false, otherwise the model will attempt to generate using the KV cache + length=1 inputs; this is more efficient, but makes interventions hard. `do_sample` is off here for consistency, but you can turn it on as well.

In [12]:
hooks, _ = model._get_feature_intervention_hooks(s_french, open_ended_es_fr_intervention_tuples, freeze_attention=False)
pre_intervention_generation = [model.generate(s_french, do_sample=False, use_past_kv_cache=False, verbose=False)]
with model.hooks(hooks):
    post_intervention_generation = [model.generate(s_french, do_sample=False, use_past_kv_cache=False, verbose=False)]

display_generations_comparison(s_french, pre_intervention_generation, post_intervention_generation)